# Ensemble Features - KPCA-MNL

In [1]:
import torch
import torch.nn as nn
from model.resnet import resnet18
from model.inception import inception_v3
from model.vgg import vgg16
from model.alexnet import alexnet
from data_loader.data_loader import get_data_loader
from model.utils import LayerActivations
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
model1 = vgg16(pretrained=False, num_classes=2)
model2 = resnet18(pretrained=False, num_classes=2)
model3 = alexnet(pretrained=False, num_classes=2)

In [4]:
# load checkpoint
model1 = model1.to(device)
best_vgg = torch.load('/data/Projects/repo/ants-and-bees-pytorch/outputs/best_model_vgg.pth')
model1.load_state_dict(best_vgg)

model2 = model2.to(device)
best_resnet = torch.load('/data/Projects/repo/ants-and-bees-pytorch/outputs/best_model_resnet.pth')
model2.load_state_dict(best_resnet)

model3 = model3.to(device)
best_alex = torch.load('/data/Projects/repo/ants-and-bees-pytorch/outputs/best_model_alex.pth')
model3.load_state_dict(best_alex)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
model1 = nn.Sequential(*list(model1.children())[:-2])

In [8]:
model2 = nn.Sequential(*list(model2.children())[:-2])

In [10]:
model3 = nn.Sequential(*list(model3.children())[:-1])

In [5]:
data_loader = get_data_loader()

In [ ]:
# Extract features for VGG16
trn_labels = []
trn_vgg_features = []
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out = model1(d)
    out = out.view(out.size(0), -1)
    trn_labels.extend(la)
    trn_vgg_features.extend(out.cpu().data.numpy().tolist())

val_labels = []
val_vgg_features = []
for d, la in data_loader['val']:
    d = d.to(device)
    with torch.no_grad():
        out = model1(d)
    out = out.view(out.size(0), -1)
    val_labels.extend(la)
    val_vgg_features.extend(out.cpu().data.numpy().tolist())

In [ ]:
# Extract features for ResNet
trn_resnet_features = []
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out = model2(d)
    out = out.view(out.size(0), -1)
    trn_resnet_features.extend(out.cpu().data.numpy().tolist())

val_resnet_features = []
for d, la in data_loader['val']:
    d = d.to(device)
    with torch.no_grad():
        out = model2(d)
    out = out.view(out.size(0), -1)
    val_resnet_features.extend(out.cpu().data.numpy().tolist())

In [ ]:
# Extract features for Alexnet
trn_alex_features = []
for d, la in data_loader['train']:
    d = d.to(device)
    with torch.no_grad():
        out = model3(d)
    out = out.view(out.size(0), -1)
    trn_alex_features.extend(out.cpu().data.numpy().tolist())

val_alex_features = []
for d, la in data_loader['val']:
    d = d.to(device)
    with torch.no_grad():
        out = model3(d)
    out = out.view(out.size(0), -1)
    val_alex_features.extend(out.cpu().data.numpy().tolist())

In [ ]:
class FeaturesDataset(Dataset):
    def __init__(self,featlst1,featlst2,featlst3,labellst):
        self.featlst1 = featlst1
        self.featlst2 = featlst2
        self.featlst3 = featlst3
        self.labellst = labellst

    def __getitem__(self,index):
        return (self.featlst1[index],self.featlst2[index],self.featlst3[index],self.labellst[index])

    def __len__(self):
        return len(self.labellst)

In [ ]:
trn_ensm_dset = FeaturesDataset(trn_vgg_features, trn_resnet_features, trn_alex_features, trn_labels)
val_ensm_dset = FeaturesDataset(val_vgg_features, val_resnet_features, val_alex_features, val_labels)

trn_ensm_loader = DataLoader(trn_ensm_dset,batch_size=8,shuffle=True)
val_ensm_loader = DataLoader(val_ensm_dset,batch_size=8)

In [ ]:
data = next(iter(trn_ensm_loader))

In [ ]:
f1, f2, f3, la = data

In [ ]:
f1